In [11]:
import pandas as pd
from thefuzz import process
import matplotlib.pyplot as plt
import numpy as np

In [12]:
similarity_threshold = 95

In [13]:
all_scores = pd.read_csv("university_scores.csv")
times_scores = pd.read_csv("times_university_scores.csv")
CWUR_scores = pd.read_csv("CWUR_university_scores.csv")
QS_scores = pd.read_csv("QS_university_scores.csv")
ARWU_scores = pd.read_csv("ARWU_university_scores.csv")
webometrics_scores = pd.read_csv("webometrics_university_scores.csv")

times_names = times_scores['University Name'].to_list()
CWUR_names = CWUR_scores['University Name'].to_list()
QS_names = QS_scores['University Name'].to_list()
ARWU_names = ARWU_scores['University Name'].to_list()
webometrics_names = webometrics_scores['University Name'].to_list()

names_list = [times_names, CWUR_names, QS_names, ARWU_names, webometrics_names]

In [14]:
similarity_indexes = []

In [15]:
def check_and_add_match(name_to_find, match, list_name, original_index, target_index, similarity_threshold = 95):
    if match[1] > similarity_threshold:
        row = {
            'original_name': name_to_find, 
            'matched_name': match[0], 
            'similarity_score': match[1],
            'list_name': list_name,
            'original_index': original_index,
            'target_index': target_index
        }
        similarity_indexes.append(row)
        
def process_list(original_name_list, target_name_list, list_name):
    for index, name_to_find in enumerate(original_name_list):
        match = process.extractOne(name_to_find, target_name_list)
        match_index = target_name_list.index(match[0])
        check_and_add_match(name_to_find, match, list_name, index, match_index, similarity_threshold)

In [16]:
print("Processing CWUR names")
process_list(times_names, CWUR_names, 'CWUR')
print("Processing QS names")
process_list(times_names, QS_names, 'QS')
print("Processing ARWU names")
process_list(times_names, ARWU_names, 'ARWU')
print("Processing Webometrics names")
process_list(times_names, webometrics_names, 'Webometrics')

Processing CWUR names
Processing QS names
Processing ARWU names
Processing Webometrics names


In [17]:
df = pd.DataFrame(similarity_indexes)
df.to_csv("similarity_index.csv", index=False)

In [18]:
grouped_df = pd.DataFrame(columns=['University Name', 'original_index', 'Times Ranking', 'CWUR Ranking', 'QS Ranking', 'ARWU Ranking', 'Webometrics Ranking'])

In [19]:
def get_ranking(list_name, index):
    if list_name == 'Times':
        score = times_scores.iloc[index][1]
    elif list_name == 'CWUR':
        score = CWUR_scores.iloc[index][1]
    elif list_name == 'QS':
        score = QS_scores.iloc[index][1]
    elif list_name == 'ARWU':
        score = ARWU_scores.iloc[index][1]
    elif list_name == 'Webometrics':
        score = webometrics_scores.iloc[index][1]
    else:
        print("List not recognized")
    return score

for index, row in df.iterrows():
    original_name = row['original_name']
    list_name = row['list_name']
    original_index = row['original_index']
    target_index = row['target_index']
    
    original_ranking = get_ranking('Times', original_index)
    ranking = get_ranking(list_name, target_index)

    if original_index in grouped_df['original_index']:
        grouped_df.loc[grouped_df['original_index'] == original_index, f'{list_name} Ranking'] = ranking
    else:
        if list_name == 'CWUR':
            new_row = [original_name, original_index, original_ranking, ranking, np.nan, np.nan, np.nan]
        elif list_name == 'QS':
            new_row = [original_name, original_index, original_ranking, np.nan, ranking, np.nan, np.nan]
        elif list_name == 'ARWU':
            new_row = [original_name, original_index, original_ranking, np.nan, np.nan, ranking, np.nan]
        elif list_name == 'Webometrics':
            new_row = [original_name, original_index, original_ranking, np.nan, np.nan, np.nan, ranking]
        grouped_df.loc[len(grouped_df)] = new_row

In [20]:
grouped_df

,University Name,original_index,Times Ranking,CWUR Ranking,QS Ranking,ARWU Ranking,Webometrics Ranking
0,University of Oxford,0,1,5.0,4,7,5.0
1,Harvard University,1,2,1.0,5,1,1.0
2,University of Cambridge,2,3,4.0,2,4,12.0
3,Stanford University,3,3,3.0,3,2,2.0
4,Massachusetts Institute of Technology,4,5,2.0,NaN,NaN,3.0
...,...,...,...,...,...,...,...
1618,West Virginia University,2218,Reporter,NaN,NaN,NaN,316.0
1619,University of Worcester,2220,Reporter,NaN,NaN,NaN,3331.0
1620,Writtle University College,2224,Reporter,NaN,NaN,NaN,8286.0
1621,Ziauddin University,2232,Reporter,NaN,NaN,NaN,4327.0


In [21]:
grouped_df.to_csv("grouped_university_ranking.csv", index=False)